# Exercise 4: Multi-Agent Handoff — Solution

Complete implementation of the multi-agent handoff workflow with Temporal orchestration.

## Prerequisites

Confirm the Temporal dev server is running and `.env` is configured.

In [ ]:
%pip install --quiet temporalio openai rich

import json
import os
import asyncio
import uuid
from datetime import timedelta
from openai import OpenAI
from rich.console import Console
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.common import RetryPolicy
from temporalio.worker import Worker, UnsandboxedWorkflowRunner

# Shared console instance for consistent workshop output
console = Console()


@activity.defn
async def get_weather(location: str) -> str:
    """Return mock weather details for the requested location."""
    # Provide canned weather data for known locations
    weather_data = {
        "San Francisco": "sunny, 72°F",
        "New York": "cloudy, 65°F",
        "London": "rainy, 58°F",
        "Tokyo": "clear, 70°F",
    }
    # Select the known weather or fall back to a default description
    weather = weather_data.get(location, "partly cloudy, 68°F")
    # Return a formatted weather string consumed by the LLM
    return f"The weather in {location} is {weather}"


@activity.defn
async def get_time(location: str) -> str:
    """Return mock local time for the requested location."""
    # Provide canned time data for known locations
    time_data = {
        "San Francisco": "10:45 AM PST",
        "New York": "1:45 PM EST",
        "London": "6:45 PM GMT",
        "Tokyo": "3:45 AM JST",
    }
    # Select the known time or fall back to a neutral value
    current_time = time_data.get(location, "12:00 PM")
    # Return a formatted time string consumed by the LLM
    return f"The current time in {location} is {current_time}"


@activity.defn
async def triage_query(query: str) -> str:
    """Classify the incoming query to the appropriate specialist agent."""
    activity.logger.info("🔍 Triaging query: %s", query)
    # Instantiate the OpenAI client using the configured API key
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    # Request the classifier model to pick the best agent route
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a query classifier. Classify queries into one of: "
                    "'weather_agent', 'time_agent', or 'general_agent'. "
                    "Respond with ONLY the agent name, nothing else."
                ),
            },
            {"role": "user", "content": query},
        ],
    )

    # Read the classifier decision from the model response
    agent_name = response.choices[0].message.content.strip()
    activity.logger.info("   → Routed to: %s", agent_name)
    # Return the target agent name to the workflow
    return agent_name


def _safe_parse_arguments(arg_string: str) -> dict:
    # Attempt to parse tool call arguments as JSON
    try:
        return json.loads(arg_string)
    except json.JSONDecodeError:
        # Default to empty args if parsing fails so the agent can continue
        activity.logger.warning("Failed to parse tool arguments, raw input: %s", arg_string)
        return {}


@activity.defn
async def weather_agent(query: str, context: dict) -> str:
    """Handle weather-related queries with tool support."""
    activity.logger.info("🌤️  Weather agent handling query")
    # Instantiate the OpenAI client for the weather specialist
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    # Describe the supported weather tool for the LLM
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get current weather for a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City name",
                        }
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    # Seed the conversation with system and user messages
    messages = [
        {"role": "system", "content": "You are a weather specialist assistant."},
        {"role": "user", "content": query},
    ]

    # Ask the model to respond and potentially call the weather tool
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
    )

    # Extract the first message object from the model output
    response_message = response.choices[0].message
    # Capture any requested tool calls from the response
    tool_calls = response_message.tool_calls

    # Handle tool calls when the model asked for external data
    if tool_calls:
        # Include the model response before executing tools
        messages.append(response_message)
        # Iterate through each tool call to service the request
        for tool_call in tool_calls:
            # Parse the tool arguments safely
            payload = _safe_parse_arguments(tool_call.function.arguments)
            # Only invoke the tool when a location argument is present
            if tool_call.function.name == "get_weather" and payload.get("location"):
                # Await the async weather activity to get mock data
                tool_result = await get_weather(**payload)
                # Attach the tool response so the LLM can use it
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_result,
                    }
                )

        # Ask the model to produce the final natural language answer
        follow_up = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        # Use the follow-up content as the final agent reply
        final_message = follow_up.choices[0].message.content
    else:
        # When no tool was called, return the original model content
        final_message = response_message.content

    # Provide the resolved answer back to the workflow
    activity.logger.info("✅ Weather agent completed")
    return final_message


@activity.defn
async def time_agent(query: str, context: dict) -> str:
    """Handle time-related queries with tool support."""
    activity.logger.info("🕐 Time agent handling query")
    # Instantiate the OpenAI client for the time specialist
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    # Describe the supported time tool for the LLM
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_time",
                "description": "Get current time for a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City name",
                        }
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    # Seed the conversation with system and user messages
    messages = [
        {"role": "system", "content": "You are a time specialist assistant."},
        {"role": "user", "content": query},
    ]

    # Ask the model to respond and potentially call the time tool
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
    )

    # Extract the first message object from the model output
    response_message = response.choices[0].message
    # Capture any requested tool calls from the response
    tool_calls = response_message.tool_calls

    # Handle tool calls when the model asked for external data
    if tool_calls:
        # Include the model response before executing tools
        messages.append(response_message)
        # Iterate through each tool call to service the request
        for tool_call in tool_calls:
            # Parse the tool arguments safely
            payload = _safe_parse_arguments(tool_call.function.arguments)
            # Only invoke the tool when a location argument is present
            if tool_call.function.name == "get_time" and payload.get("location"):
                # Await the async time activity to get mock data
                tool_result = await get_time(**payload)
                # Attach the tool response so the LLM can use it
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_result,
                    }
                )

        # Ask the model to produce the final natural language answer
        follow_up = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        # Use the follow-up content as the final agent reply
        final_message = follow_up.choices[0].message.content
    else:
        # When no tool was called, return the original model content
        final_message = response_message.content

    # Provide the resolved answer back to the workflow
    activity.logger.info("✅ Time agent completed")
    return final_message


@workflow.defn
class MultiAgentWorkflow:
    """Workflow orchestrating triage and specialist agent hand-offs."""

    @workflow.run
    async def run(self, query: str, trace_id: str) -> str:
        # Note that the workflow is starting execution
        workflow.logger.info("🚀 Multi-agent workflow started")
        # Record the incoming end-user query for traceability
        workflow.logger.info("   Query: %s", query)
        # Record the trace identifier for cross-system debugging
        workflow.logger.info("   Trace ID: %s", trace_id)

        # Track minimal conversation context for future enhancements
        context = {"history": [], "trace_id": trace_id}

        # Ask the triage activity to determine the best agent for the query
        agent_to_use = await workflow.execute_activity(
            triage_query,
            args=[query],
            start_to_close_timeout=timedelta(seconds=10),
            retry_policy=RetryPolicy(
                maximum_attempts=3,
                initial_interval=timedelta(seconds=1),
            ),
        )

        # Log which agent will receive the request next
        workflow.logger.info("   Routing to: %s", agent_to_use)

        # Invoke the appropriate specialist based on the triage decision
        if agent_to_use == "weather_agent":
            # Execute the weather specialist with retry safety
            result = await workflow.execute_activity(
                weather_agent,
                args=[query, context],
                start_to_close_timeout=timedelta(seconds=30),
                retry_policy=RetryPolicy(maximum_attempts=3),
            )
        elif agent_to_use == "time_agent":
            # Execute the time specialist with retry safety
            result = await workflow.execute_activity(
                time_agent,
                args=[query, context],
                start_to_close_timeout=timedelta(seconds=30),
                retry_policy=RetryPolicy(maximum_attempts=3),
            )
        else:
            # Provide a placeholder when no specific specialist exists
            result = "Query routed to general agent (not implemented in this exercise)"

        # Confirm completion before returning to the caller
        workflow.logger.info("✅ Multi-agent workflow completed")
        return result


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Run the Solution

Execute the solution workflow to observe agent handoffs.

In [7]:
# Run the multi-agent handoff solution
async def run_solution() -> None:
    # Introduce the demo execution in the notebook
    console.print("\n[bold cyan]🚀 Exercise 4: Multi-Agent Handoff — Solution[/bold cyan]\n")

    # Create a unique identifier to correlate workflow and traces
    trace_id = str(uuid.uuid4())
    # Display the trace identifier for manual debugging
    console.print(f"[yellow]Trace ID:[/yellow] {trace_id}\n")

    # Establish a connection to the local Temporal server
    client = await Client.connect("localhost:7233")
    # Specify the worker task queue dedicated to this example
    task_queue = "multi-agent-queue"

    # Start a Temporal worker hosting the workflow and activities
    async with Worker(
        client,
        task_queue=task_queue,
        workflows=[MultiAgentWorkflow],
        activities=[triage_query, weather_agent, time_agent, get_weather, get_time],
        workflow_runner=UnsandboxedWorkflowRunner(),
        debug_mode=True,
    ):
        # Seed the example with a combined weather and time question
        query = "What's the weather like in London and what time is it there?"
        # Derive a workflow ID that embeds the trace reference
        workflow_id = f"multi-agent-{trace_id}"

        # Display the query being routed through the workflow
        console.print(f"[yellow]Query:[/yellow] {query}\n")

        # Execute the workflow and wait for its final response
        result = await client.execute_workflow(
            MultiAgentWorkflow.run,
            args=[query, trace_id],
            id=workflow_id,
            task_queue=task_queue,
        )

    # Show the natural language outcome returned by the agent chain
    console.print(f"\n[bold green]✅ Final Response:[/bold green]\n{result}\n")
    # Point operators to the Temporal UI entry for deeper inspection
    console.print(
        f"[yellow]View in Temporal UI:[/yellow] "
        f"http://localhost:8233/namespaces/default/workflows/{workflow_id}"
    )
    # Echo the trace ID again for quick copy/paste access
    console.print(f"[yellow]Trace ID:[/yellow] {trace_id}\n")

# Detect whether the notebook already has an active event loop
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    # No loop means we can safely drive execution with asyncio.run
    asyncio.run(run_solution())
else:
    # Apply nest_asyncio so we can reuse the existing notebook loop
    import nest_asyncio
    nest_asyncio.apply()
    # Schedule the solution coroutine on the current loop
    task = loop.create_task(run_solution())
    # Ensure the scheduled task completes before moving on
    await task

🚀 Exercise 4: Multi-Agent Handoff — Solution

Trace ID: bf7e4a3f-1c60-4d91-9540-8a486796a9ab

Query: What's the weather like in London and what time is it there?

✅ Final Response:
Query routed to general agent (not implemented in this exercise)

View in Temporal UI: 
http://localhost:8233/namespaces/default/workflows/multi-agent-bf7e4a3f-1c60-4d91-9540-8a486796a9ab

Trace ID: bf7e4a3f-1c60-4d91-9540-8a486796a9ab

## Source Code

The workflow orchestration and activity implementations are defined above in the first code cell.